# Load packages

In [6]:
# skip reinstalling packages we already have
using Pkg

pkgs = [
    "MLJ", "MLJBase", "MLJModels", "MLJEnsembles", "MLJLinearModels",
    "DecisionTree", "MLJDecisionTreeInterface", "NaiveBayes", 
    "MLJNaiveBayesInterface", "EvoTrees", "CategoricalArrays", "Random",
    "LIBSVM", "MLJLIBSVMInterface", "Plots", "MLJModelInterface",
    "CSV", "DataFrames", "UrlDownload", "XGBoost", "NNlib"
]

# Filter out packages already installed
missing_pkgs = filter(pkg -> !(pkg in keys(Pkg.project().dependencies)), pkgs)

if !isempty(missing_pkgs)
    println("Installing missing packages: ", missing_pkgs)
    Pkg.add(missing_pkgs)
else
    println(" All required packages are already installed.")
end


 All required packages are already installed.


In [10]:
using MLJ
using MLJBase
using LIBSVM
using NNlib
using Flux
using Flux.Losses
using Statistics

[ Info: Precompiling MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7] 


In [3]:
# Load PCA
PCA_model = MLJ.@load PCA pkg="MultivariateStats"

[ Info: For silent loading, specify `verbosity=0`. 


import MLJMultivariateStatsInterface ✔


MLJMultivariateStatsInterface.PCA

In [7]:
#Load your library of functions
include("utils.jl")
# Set a global random seed for reproducibility
using Random
Random.seed!(42)

TaskLocalRNG()

# Load Data

In [8]:
using CSV, DataFrames, Random
using CategoricalArrays

df = CSV.read("./data/updated_pollution_dataset.csv", DataFrame)

# Some log
println("First 5 rows of df:")
show(df[1:5, :], allcols=true)


# Convert last column to categorical (in-place!)
df[!, end] = categorical(df[!, end])

# Extract the integer codes of the categories
targets = Float32.(levelcode.(df[!, end]))

# Use all columns except the last one as inputs
inputs = Matrix{Float32}(df[:, 1:end-1])

println("First 5 inputs::")
for i in 1:5
    println(inputs[i, :])
end

println("\n\nFirst 5 targets:")
println(targets[1:5])

# Extract labels (categories) as strings
label_names = levels(df[!, 10])
println("Labels: ", label_names)

First 5 rows of df:
5×10 DataFrame
 Row │ Temperature  Humidity  PM2.5    PM10     NO2      SO2      CO       Proximity_to_Industrial_Areas  Population_Density  Air Quality 
     │ Float64      Float64   Float64  Float64  Float64  Float64  Float64  Float64                        Int64               String15    
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        29.8      59.1      5.2     17.9     18.9      9.2     1.72                            6.3                 319  Moderate
   2 │        28.3      75.6      2.3     12.2     30.8      9.7     1.64                            6.0                 611  Moderate
   3 │        23.1      74.7     26.7     33.8     24.4     12.6     1.63                            5.2                 619  Moderate
   4 │        27.1      39.1      6.1      6.3     13.5      5.3     1.15                           11.1                 551  Good
   5 │      

In [6]:
results = Dict()
dimsPCA = [6,7,8]
crossValidationIndices = crossvalidation(targets, 5)

5000-element Vector{Int64}:
 1
 3
 5
 4
 1
 3
 3
 5
 1
 1
 5
 1
 2
 ⋮
 3
 4
 4
 4
 5
 1
 2
 4
 4
 3
 5
 1

In [7]:
function printExperimentResult(model, hyperparams, dimPCA, results, class_labels)
    (
        (accuracy_mean, accuracy_std),
        (error_rate_mean, error_rate_std),
        (sensitivity_mean, sensitivity_std),
        (specificity_mean, specificity_std),
        (ppv_mean, ppv_std),
        (npv_mean, npv_std),
        (f1_mean, f1_std),
        cm
    ) = results

    println("\n=====================================================")
    println(" Model: $model | PCA outdim: $dimPCA")
    println(" Hyperparameters: $hyperparams")
    println("=====================================================")

    println(" Accuracy (mean)               : ", round(accuracy_mean, digits=4))
    println(" Accuracy (std)                : ", round(accuracy_std, digits=4))

    println(" Error Rate (mean)             : ", round(error_rate_mean, digits=4))
    println(" Error Rate (std)              : ", round(error_rate_std, digits=4))

    println(" Sensitivity/Recall (mean)     : ", round(sensitivity_mean, digits=4))
    println(" Sensitivity/Recall (std)      : ", round(sensitivity_std,  digits=4))

    println(" Specificity (mean)            : ", round(specificity_mean, digits=4))
    println(" Specificity (std)             : ", round(specificity_std,  digits=4))

    println(" PPV (mean)                    : ", round(ppv_mean,         digits=4))
    println(" PPV (std)                     : ", round(ppv_std,          digits=4))

    println(" NPV (mean)                    : ", round(npv_mean,         digits=4))
    println(" NPV (std)                     : ", round(npv_std,          digits=4))

    println(" F1 Score (mean)               : ", round(f1_mean,          digits=4))
    println(" F1 Score (std)                : ", round(f1_std,           digits=4))

    println("\nConfusion Matrix:")
    println(cm)

    PrettyTables.pretty_table(DataFrame(cm, :auto); header=class_labels, row_labels=class_labels)

    println("=====================================================\n")
end


printExperimentResult (generic function with 1 method)

# Artificial Neural Networks

In [8]:
############# 1. ARTIFICIAL NEURAL NETWORKS (8+ topologies) #############
default_ann = Dict(      
    "numExecutions" => 5,
    #"transferFunctions" => [σ, σ, σ, σ],
    "maxEpochs" => 200,
    "minLoss" => 0.0,
    "learningRate" => 0.01,
    "validationRatio" => 0.1,
    "maxEpochsVal" => 20
)

ann_search_space = [
    Dict("topology"=>[4, 4]),
    Dict("topology"=>[8, 8]),
    Dict("topology"=>[16, 16]),
    Dict("topology"=>[10, 4]),
    Dict("topology"=>[10, 6, 4]),
    Dict("topology"=>[10, 8, 4]),
    Dict("topology"=>[10, 8, 6, 4]),
    Dict("topology"=>[10, 12, 6, 4])
]

8-element Vector{Dict{String, Vector{Int64}}}:
 Dict("topology" => [4, 4])
 Dict("topology" => [8, 8])
 Dict("topology" => [16, 16])
 Dict("topology" => [10, 4])
 Dict("topology" => [10, 6, 4])
 Dict("topology" => [10, 8, 4])
 Dict("topology" => [10, 8, 6, 4])
 Dict("topology" => [10, 12, 6, 4])

In [9]:
########################
# 1. ANN GRID SEARCH
########################
ann_results = []

for hp in ann_search_space
    for dim in dimsPCA
        println("\n=== ANN experiment: topology = $(hp["topology"]) | PCA maxoutdim = $(dim) ===")
        full_hp = merge(default_ann, hp)
        res = modelCrossValidationPCA(:ANN, full_hp, (inputs, targets), crossValidationIndices, dim)
        push!(ann_results, (model=:ANN, hyperparams=hp, dimPCA=dim, results=res))
    end
end

results[:ANN] = ann_results


=== ANN experiment: topology = [4, 4] | PCA maxoutdim = 6 ===

=== ANN experiment: topology = [4, 4] | PCA maxoutdim = 7 ===

=== ANN experiment: topology = [4, 4] | PCA maxoutdim = 8 ===

=== ANN experiment: topology = [8, 8] | PCA maxoutdim = 6 ===

=== ANN experiment: topology = [8, 8] | PCA maxoutdim = 7 ===

=== ANN experiment: topology = [8, 8] | PCA maxoutdim = 8 ===

=== ANN experiment: topology = [16, 16] | PCA maxoutdim = 6 ===

=== ANN experiment: topology = [16, 16] | PCA maxoutdim = 7 ===

=== ANN experiment: topology = [16, 16] | PCA maxoutdim = 8 ===

=== ANN experiment: topology = [10, 4] | PCA maxoutdim = 6 ===

=== ANN experiment: topology = [10, 4] | PCA maxoutdim = 7 ===

=== ANN experiment: topology = [10, 4] | PCA maxoutdim = 8 ===

=== ANN experiment: topology = [10, 6, 4] | PCA maxoutdim = 6 ===

=== ANN experiment: topology = [10, 6, 4] | PCA maxoutdim = 7 ===

=== ANN experiment: topology = [10, 6, 4] | PCA maxoutdim = 8 ===

=== ANN experiment: topology = [1

24-element Vector{Any}:
 (model = :ANN, hyperparams = Dict("topology" => [4, 4]), dimPCA = 6, results = ((0.94048005f0, 0.007690985f0), (0.05952f0, 0.0076910355f0), (0.94048005f0, 0.007690985f0), (0.9826414f0, 0.0017484212f0), (0.94006604f0, 0.0076740775f0), (0.98477715f0, 0.0020668781f0), (0.94012755f0, 0.007616604f0), Float32[332.28 0.71999997 6.6 0.4; 0.6 243.96 8.2 7.2399993; 11.96 8.12 175.48001 4.44; 0.4 5.12 5.72 188.76]))
 (model = :ANN, hyperparams = Dict("topology" => [4, 4]), dimPCA = 7, results = ((0.93992007f0, 0.0068317014f0), (0.060080003f0, 0.006831693f0), (0.93992007f0, 0.0068317014f0), (0.9822475f0, 0.0011926473f0), (0.93978006f0, 0.0071935887f0), (0.98485124f0, 0.0018926415f0), (0.9393891f0, 0.0066998834f0), Float32[332.76 0.76 6.0799994 0.4; 0.64 243.8 8.280001 7.28; 12.559999 7.96 175.92001 3.56; 0.44 5.16 6.96 187.44]))
 (model = :ANN, hyperparams = Dict("topology" => [4, 4]), dimPCA = 8, results = ((0.93776f0, 0.010603237f0), (0.062240005f0, 0.010603208f0), (0.93

In [10]:
for entry in results[:ANN]
    printExperimentResult(entry.model, entry.hyperparams, entry.dimPCA, entry.results, label_names)
end


 Model: ANN | PCA outdim: 6
 Hyperparameters: Dict("topology" => [4, 4])
 Accuracy (mean)               : 0.9405
 Accuracy (std)                : 0.0077
 Error Rate (mean)             : 0.0595
 Error Rate (std)              : 0.0077
 Sensitivity/Recall (mean)     : 0.9405
 Sensitivity/Recall (std)      : 0.0077
 Specificity (mean)            : 0.9826
 Specificity (std)             : 0.0017
 PPV (mean)                    : 0.9401
 PPV (std)                     : 0.0077
 NPV (mean)                    : 0.9848
 NPV (std)                     : 0.0021
 F1 Score (mean)               : 0.9401
 F1 Score (std)                : 0.0076

Confusion Matrix:
Float32[332.28 0.71999997 6.6 0.4; 0.6 243.96 8.2 7.2399993; 11.96 8.12 175.48001 4.44; 0.4 5.12 5.72 188.76]
┌───────────┬────────┬───────────┬──────────┬────────┐
│           │   Good │ Hazardous │ Moderate │   Poor │
├───────────┼────────┼───────────┼──────────┼────────┤
│      Good │ 332.28 │      0.72 │      6.6 │    0.4 │
│ Hazardous │    

# Support Vector Machines

In [13]:
SVMClassifier = MLJ.@load SVC pkg=LIBSVM verbosity=0

MLJLIBSVMInterface.SVC

In [14]:
############# 2. SVM (8+ configs: kernels × C) #############
default_svm = Dict(
    "gamma" => 1.0,
    "degree" => 3,
    "coef0" => 0.0
)

svm_search_space = [
    Dict("kernel"=>"linear",  "C"=>0.1),
    Dict("kernel"=>"linear",  "C"=>1.0),
    Dict("kernel"=>"linear",  "C"=>10.0),
    Dict("kernel"=>"rbf",     "C"=>1.0, "gamma"=>2.0),
    Dict("kernel"=>"rbf",     "C"=>10.0,"gamma"=>0.5),
    Dict("kernel"=>"sigmoid", "C"=>1.0, "gamma"=>1.0),
    Dict("kernel"=>"poly",    "C"=>1.0, "degree"=>3, "gamma"=>1.0),
    Dict("kernel"=>"poly",    "C"=>5.0, "degree"=>4, "gamma"=>0.5),
]

8-element Vector{Dict{String, Any}}:
 Dict("C" => 0.1, "kernel" => "linear")
 Dict("C" => 1.0, "kernel" => "linear")
 Dict("C" => 10.0, "kernel" => "linear")
 Dict("C" => 1.0, "kernel" => "rbf", "gamma" => 2.0)
 Dict("C" => 10.0, "kernel" => "rbf", "gamma" => 0.5)
 Dict("C" => 1.0, "kernel" => "sigmoid", "gamma" => 1.0)
 Dict("C" => 1.0, "kernel" => "poly", "gamma" => 1.0, "degree" => 3)
 Dict("C" => 5.0, "kernel" => "poly", "gamma" => 0.5, "degree" => 4)

In [15]:
########################
# 2. SVM GRID SEARCH
########################
svm_results = []

for hp in svm_search_space
    for dim in dimsPCA
        println("\n=== SVM experiment: kernel=$(hp["kernel"]) C=$(get(hp,"C","-")) | PCA maxoutdim = $(dim) ===")
        full_hp = merge(default_svm, hp)
        res = modelCrossValidationPCA(:SVC, full_hp, (inputs, targets), crossValidationIndices, dim)
        push!(svm_results, (model=:SVC, hyperparams=hp, dimPCA=dim, results=res))
    end
end

results[:SVC] = svm_results


=== SVM experiment: kernel=linear C=0.1 | PCA maxoutdim = 6 ===

=== SVM experiment: kernel=linear C=0.1 | PCA maxoutdim = 7 ===

=== SVM experiment: kernel=linear C=0.1 | PCA maxoutdim = 8 ===

=== SVM experiment: kernel=linear C=1.0 | PCA maxoutdim = 6 ===

=== SVM experiment: kernel=linear C=1.0 | PCA maxoutdim = 7 ===

=== SVM experiment: kernel=linear C=1.0 | PCA maxoutdim = 8 ===

=== SVM experiment: kernel=linear C=10.0 | PCA maxoutdim = 6 ===

=== SVM experiment: kernel=linear C=10.0 | PCA maxoutdim = 7 ===

=== SVM experiment: kernel=linear C=10.0 | PCA maxoutdim = 8 ===

=== SVM experiment: kernel=rbf C=1.0 | PCA maxoutdim = 6 ===

=== SVM experiment: kernel=rbf C=1.0 | PCA maxoutdim = 7 ===

=== SVM experiment: kernel=rbf C=1.0 | PCA maxoutdim = 8 ===

=== SVM experiment: kernel=rbf C=10.0 | PCA maxoutdim = 6 ===

=== SVM experiment: kernel=rbf C=10.0 | PCA maxoutdim = 7 ===

=== SVM experiment: kernel=rbf C=10.0 | PCA maxoutdim = 8 ===

=== SVM experiment: kernel=sigmoid C

24-element Vector{Any}:
 (model = :SVC, hyperparams = Dict{String, Any}("C" => 0.1, "kernel" => "linear"), dimPCA = 6, results = ((0.9195999f0, 0.009423383f0), (0.080400005f0, 0.009423374f0), (0.9195999f0, 0.009423383f0), (0.9710825f0, 0.0037891099f0), (0.9199994f0, 0.008790069f0), (0.9810745f0, 0.0022209468f0), (0.91775674f0, 0.009813511f0), Float32[330.2 4.0 4.2 1.6; 0.2 240.2 10.6 9.0; 19.2 9.0 170.0 1.8; 2.4 7.2 11.2 179.2]))
 (model = :SVC, hyperparams = Dict{String, Any}("C" => 0.1, "kernel" => "linear"), dimPCA = 7, results = ((0.92080003f0, 0.010686428f0), (0.0792f0, 0.01068644f0), (0.92080003f0, 0.010686428f0), (0.97122383f0, 0.0043843295f0), (0.9211836f0, 0.010112118f0), (0.98120815f0, 0.0025284868f0), (0.91903895f0, 0.011139641f0), Float32[330.2 4.0 4.2 1.6; 0.2 241.0 9.8 9.0; 19.8 8.8 169.6 1.8; 2.6 6.8 10.6 180.0]))
 (model = :SVC, hyperparams = Dict{String, Any}("C" => 0.1, "kernel" => "linear"), dimPCA = 8, results = ((0.92480004f0, 0.009444583f0), (0.0752f0, 0.009444576

In [16]:
for entry in results[:SVC]
    printExperimentResult(entry.model, entry.hyperparams, entry.dimPCA, entry.results, label_names)
end


 Model: SVC | PCA outdim: 6
 Hyperparameters: Dict{String, Any}("C" => 0.1, "kernel" => "linear")
 Accuracy (mean)               : 0.9196
 Accuracy (std)                : 0.0094
 Error Rate (mean)             : 0.0804
 Error Rate (std)              : 0.0094
 Sensitivity/Recall (mean)     : 0.9196
 Sensitivity/Recall (std)      : 0.0094
 Specificity (mean)            : 0.9711
 Specificity (std)             : 0.0038
 PPV (mean)                    : 0.92
 PPV (std)                     : 0.0088
 NPV (mean)                    : 0.9811
 NPV (std)                     : 0.0022
 F1 Score (mean)               : 0.9178
 F1 Score (std)                : 0.0098

Confusion Matrix:
Float32[330.2 4.0 4.2 1.6; 0.2 240.2 10.6 9.0; 19.2 9.0 170.0 1.8; 2.4 7.2 11.2 179.2]
┌───────────┬───────┬───────────┬──────────┬───────┐
│           │  Good │ Hazardous │ Moderate │  Poor │
├───────────┼───────┼───────────┼──────────┼───────┤
│      Good │ 330.2 │       4.0 │      4.2 │   1.6 │
│ Hazardous │   0.2 │    

# Decission Trees

In [41]:
DTClassifier = MLJ.@load DecisionTreeClassifier pkg=DecisionTree verbosity=0

MLJDecisionTreeInterface.DecisionTreeClassifier

In [42]:
############# 3. DECISION TREES (6 depths) #############
default_dt = Dict(
    "rng" => Random.MersenneTwister(1)
)

dt_search_space = [
    Dict("max_depth"=>2),
    Dict("max_depth"=>3),
    Dict("max_depth"=>4),
    Dict("max_depth"=>5),
    Dict("max_depth"=>6),
    Dict("max_depth"=>8)
]

6-element Vector{Dict{String, Int64}}:
 Dict("max_depth" => 2)
 Dict("max_depth" => 3)
 Dict("max_depth" => 4)
 Dict("max_depth" => 5)
 Dict("max_depth" => 6)
 Dict("max_depth" => 8)

In [43]:
########################
# 3. DECISION TREE GRID SEARCH
########################
dt_results = []

for hp in dt_search_space
    for dim in dimsPCA
        println("\n=== Decision Tree experiment: max_depth=$(hp["max_depth"]) | PCA maxoutdim = $(dim) ===")
        full_hp = merge(default_dt, hp)
        res = modelCrossValidationPCA(:DecisionTreeClassifier, full_hp, (inputs, targets), crossValidationIndices, dim)
        push!(dt_results, (model=:DT, hyperparams=hp, dimPCA=dim, results=res))
    end
end

results[:DT] = dt_results


=== Decision Tree experiment: max_depth=2 | PCA maxoutdim = 6 ===

=== Decision Tree experiment: max_depth=2 | PCA maxoutdim = 7 ===

=== Decision Tree experiment: max_depth=2 | PCA maxoutdim = 8 ===

=== Decision Tree experiment: max_depth=3 | PCA maxoutdim = 6 ===

=== Decision Tree experiment: max_depth=3 | PCA maxoutdim = 7 ===

=== Decision Tree experiment: max_depth=3 | PCA maxoutdim = 8 ===

=== Decision Tree experiment: max_depth=4 | PCA maxoutdim = 6 ===

=== Decision Tree experiment: max_depth=4 | PCA maxoutdim = 7 ===

=== Decision Tree experiment: max_depth=4 | PCA maxoutdim = 8 ===

=== Decision Tree experiment: max_depth=5 | PCA maxoutdim = 6 ===

=== Decision Tree experiment: max_depth=5 | PCA maxoutdim = 7 ===

=== Decision Tree experiment: max_depth=5 | PCA maxoutdim = 8 ===

=== Decision Tree experiment: max_depth=6 | PCA maxoutdim = 6 ===

=== Decision Tree experiment: max_depth=6 | PCA maxoutdim = 7 ===

=== Decision Tree experiment: max_depth=6 | PCA maxoutdim = 8

18-element Vector{Any}:
 (model = :DT, hyperparams = Dict("max_depth" => 2), dimPCA = 6, results = ((0.83219993f0, 0.0054497677f0), (0.16780001f0, 0.0054497714f0), (0.83219993f0, 0.0054497677f0), (0.94234765f0, 0.0018211625f0), (0.7638066f0, 0.0061601675f0), (0.96546876f0, 0.0011020584f0), (0.7940912f0, 0.005148579f0), Float32[319.4 8.2 7.2 5.2; 0.8 226.8 24.6 7.8; 27.4 21.0 151.6 0.0; 3.8 21.8 40.0 134.4]))
 (model = :DT, hyperparams = Dict("max_depth" => 2), dimPCA = 7, results = ((0.83219993f0, 0.0054497677f0), (0.16780001f0, 0.0054497714f0), (0.83219993f0, 0.0054497677f0), (0.94234765f0, 0.0018211625f0), (0.7638066f0, 0.0061601675f0), (0.96546876f0, 0.0011020584f0), (0.7940912f0, 0.005148579f0), Float32[319.4 8.2 7.2 5.2; 0.8 226.8 24.6 7.8; 27.4 21.0 151.6 0.0; 3.8 21.8 40.0 134.4]))
 (model = :DT, hyperparams = Dict("max_depth" => 2), dimPCA = 8, results = ((0.83219993f0, 0.0054497677f0), (0.16780001f0, 0.0054497714f0), (0.83219993f0, 0.0054497677f0), (0.94234765f0, 0.0018211625f

In [44]:
for entry in results[:DT]
    printExperimentResult(entry.model, entry.hyperparams, entry.dimPCA, entry.results, label_names)
end


 Model: DT | PCA outdim: 6
 Hyperparameters: Dict("max_depth" => 2)
 Accuracy (mean)               : 0.8322
 Accuracy (std)                : 0.0054
 Error Rate (mean)             : 0.1678
 Error Rate (std)              : 0.0054
 Sensitivity/Recall (mean)     : 0.8322
 Sensitivity/Recall (std)      : 0.0054
 Specificity (mean)            : 0.9423
 Specificity (std)             : 0.0018
 PPV (mean)                    : 0.7638
 PPV (std)                     : 0.0062
 NPV (mean)                    : 0.9655
 NPV (std)                     : 0.0011
 F1 Score (mean)               : 0.7941
 F1 Score (std)                : 0.0051

Confusion Matrix:
Float32[319.4 8.2 7.2 5.2; 0.8 226.8 24.6 7.8; 27.4 21.0 151.6 0.0; 3.8 21.8 40.0 134.4]
┌───────────┬───────┬───────────┬──────────┬───────┐
│           │  Good │ Hazardous │ Moderate │  Poor │
├───────────┼───────┼───────────┼──────────┼───────┤
│      Good │ 319.4 │       8.2 │      7.2 │   5.2 │
│ Hazardous │   0.8 │     226.8 │     24.6 │   7.8 

# K-Nearest Neighbors

In [45]:
kNNClassifier = MLJ.@load KNNClassifier pkg=NearestNeighborModels verbosity=0

NearestNeighborModels.KNNClassifier

In [46]:
############# 4. kNN (6 values) #############
knn_search_space = [
    Dict("K"=>1),
    Dict("K"=>3),
    Dict("K"=>5),
    Dict("K"=>7),
    Dict("K"=>9),
    Dict("K"=>11)
]

6-element Vector{Dict{String, Int64}}:
 Dict("K" => 1)
 Dict("K" => 3)
 Dict("K" => 5)
 Dict("K" => 7)
 Dict("K" => 9)
 Dict("K" => 11)

In [47]:
########################
# 4. KNN GRID SEARCH
########################
knn_results = []

for hp in knn_search_space
    for dim in dimsPCA
        println("\n=== kNN experiment: K=$(hp["K"]) | PCA maxoutdim = $(dim) ===")
        res = modelCrossValidationPCA(:KNeighborsClassifier, hp, (inputs, targets), crossValidationIndices, dim)
        push!(knn_results, (model=:KNN, hyperparams=hp, dimPCA=dim, results=res))
    end
end

results[:KNN] = knn_results


=== kNN experiment: K=1 | PCA maxoutdim = 6 ===

=== kNN experiment: K=1 | PCA maxoutdim = 7 ===

=== kNN experiment: K=1 | PCA maxoutdim = 8 ===

=== kNN experiment: K=3 | PCA maxoutdim = 6 ===

=== kNN experiment: K=3 | PCA maxoutdim = 7 ===

=== kNN experiment: K=3 | PCA maxoutdim = 8 ===

=== kNN experiment: K=5 | PCA maxoutdim = 6 ===

=== kNN experiment: K=5 | PCA maxoutdim = 7 ===

=== kNN experiment: K=5 | PCA maxoutdim = 8 ===

=== kNN experiment: K=7 | PCA maxoutdim = 6 ===

=== kNN experiment: K=7 | PCA maxoutdim = 7 ===

=== kNN experiment: K=7 | PCA maxoutdim = 8 ===

=== kNN experiment: K=9 | PCA maxoutdim = 6 ===

=== kNN experiment: K=9 | PCA maxoutdim = 7 ===

=== kNN experiment: K=9 | PCA maxoutdim = 8 ===

=== kNN experiment: K=11 | PCA maxoutdim = 6 ===

=== kNN experiment: K=11 | PCA maxoutdim = 7 ===

=== kNN experiment: K=11 | PCA maxoutdim = 8 ===


18-element Vector{Any}:
 (model = :KNN, hyperparams = Dict("K" => 1), dimPCA = 6, results = ((0.9094f0, 0.005319782f0), (0.0906f0, 0.005319774f0), (0.9094f0, 0.005319782f0), (0.97251666f0, 0.0018988046f0), (0.9086259f0, 0.00518439f0), (0.97641736f0, 0.00093733f0), (0.9085716f0, 0.005451446f0), Float32[327.6 2.8 8.8 0.8; 0.8 237.4 12.8 9.0; 16.2 13.6 164.2 6.0; 1.4 8.2 10.2 180.2]))
 (model = :KNN, hyperparams = Dict("K" => 1), dimPCA = 7, results = ((0.9118f0, 0.00554077f0), (0.0882f0, 0.005540759f0), (0.9118f0, 0.00554077f0), (0.9731444f0, 0.0025371849f0), (0.91048753f0, 0.0060021915f0), (0.9779686f0, 0.0012446409f0), (0.9105797f0, 0.006166984f0), Float32[329.6 1.2 8.2 1.0; 0.8 236.8 12.6 9.8; 18.4 13.2 162.4 6.0; 1.2 6.4 9.4 183.0]))
 (model = :KNN, hyperparams = Dict("K" => 1), dimPCA = 8, results = ((0.91120005f0, 0.0055856993f0), (0.0888f0, 0.0055856947f0), (0.91120005f0, 0.0055856993f0), (0.97299445f0, 0.0014615624f0), (0.90951043f0, 0.0056554712f0), (0.9785203f0, 0.0012908599f0)

In [48]:
for entry in results[:KNN]
    printExperimentResult(entry.model, entry.hyperparams, entry.dimPCA, entry.results, label_names)
end


 Model: KNN | PCA outdim: 6
 Hyperparameters: Dict("K" => 1)
 Accuracy (mean)               : 0.9094
 Accuracy (std)                : 0.0053
 Error Rate (mean)             : 0.0906
 Error Rate (std)              : 0.0053
 Sensitivity/Recall (mean)     : 0.9094
 Sensitivity/Recall (std)      : 0.0053
 Specificity (mean)            : 0.9725
 Specificity (std)             : 0.0019
 PPV (mean)                    : 0.9086
 PPV (std)                     : 0.0052
 NPV (mean)                    : 0.9764
 NPV (std)                     : 0.0009
 F1 Score (mean)               : 0.9086
 F1 Score (std)                : 0.0055

Confusion Matrix:
Float32[327.6 2.8 8.8 0.8; 0.8 237.4 12.8 9.0; 16.2 13.6 164.2 6.0; 1.4 8.2 10.2 180.2]
┌───────────┬───────┬───────────┬──────────┬───────┐
│           │  Good │ Hazardous │ Moderate │  Poor │
├───────────┼───────┼───────────┼──────────┼───────┤
│      Good │ 327.6 │       2.8 │      8.8 │   0.8 │
│ Hazardous │   0.8 │     237.4 │     12.8 │   9.0 │
│  Mod



 Model: KNN | PCA outdim: 6
 Hyperparameters: Dict("K" => 9)
 Accuracy (mean)               : 0.9342
 Accuracy (std)                : 0.0077
 Error Rate (mean)             : 0.0658
 Error Rate (std)              : 0.0077
 Sensitivity/Recall (mean)     : 0.9342
 Sensitivity/Recall (std)      : 0.0077
 Specificity (mean)            : 0.9776
 Specificity (std)             : 0.002
 PPV (mean)                    : 0.934
 PPV (std)                     : 0.0079
 NPV (mean)                    : 0.9848
 NPV (std)                     : 0.0018
 F1 Score (mean)               : 0.933
 F1 Score (std)                : 0.0079

Confusion Matrix:
Float32[332.6 2.6 3.8 1.0; 0.2 242.4 9.4 8.0; 16.2 8.4 172.8 2.6; 0.6 5.4 7.6 186.4]
┌───────────┬───────┬───────────┬──────────┬───────┐
│           │  Good │ Hazardous │ Moderate │  Poor │
├───────────┼───────┼───────────┼──────────┼───────┤
│      Good │ 332.6 │       2.6 │      3.8 │   1.0 │
│ Hazardous │   0.2 │     242.4 │      9.4 │   8.0 │
│  Moderate

# Stacking Ensemble

In [23]:
SVMClassifier = @load ProbabilisticSVC pkg=LIBSVM verbosity=0
DTClassifier  = @load DecisionTreeClassifier pkg=DecisionTree verbosity=0
kNNClassifier = @load KNNClassifier pkg=NearestNeighborModels verbosity=0
PCA = MLJ.@load PCA pkg="MultivariateStats" verbosity=0

trainIdx, testIdx = holdOut(size(df,1), 0.2)

trainingInputs  = inputs[trainIdx, :]
testInputs      = inputs[testIdx, :]

trainingTargets = targets[trainIdx]
testTargets     = targets[testIdx]

# X as DataFrame
train_df = DataFrame(trainingInputs, :auto)
test_df  = DataFrame(testInputs, :auto)

# y as categorical
train_y_cat = categorical(trainingTargets)
test_y_cat  = categorical(testTargets)

# ---- Base Models ----
svm_pca = Pipeline(
    pca = PCA(maxoutdim=8),
    model = SVMClassifier(kernel=LIBSVM.Kernel.RadialBasis, cost=1.0, gamma=2.0)
)

dt_pca = Pipeline(
    pca = PCA(maxoutdim=8),
    model = DTClassifier(max_depth=5)
)

knn_pca = Pipeline(
    pca = PCA(maxoutdim=6),
    model = kNNClassifier(K=9)
)

# ---- Stacking Model ----
stack_model = Stack(;
    metalearner = DTClassifier(max_depth=3, rng=Random.MersenneTwister(1)),
    resampling = CV(nfolds=5, shuffle=true, rng=123),
    measure = accuracy, 
    svm = svm_pca,
    knn = knn_pca,
    dt = dt_pca
)

# Train the stacking model on your train dataset
stack_mach = machine(stack_model, train_df, train_y_cat) |> MLJ.fit!

y_pred = mode.(MLJ.predict(stack_mach, test_df))
acc = MLJ.accuracy(y_pred, test_y_cat)
println("Stack ensemble accuracy = $(round(acc*100, digits=2)) %")

Stack ensemble accuracy = 62.5 %


┌ Warning: Could not infer input_scitype of the stack
└ @ MLJBase ~/.julia/packages/MLJBase/GY2fM/src/composition/models/stacking.jl:187
[ Info: Training machine(ProbabilisticStack(metalearner = DecisionTreeClassifier(max_depth = 3, …), …), …).
[ Info: Training machine(:svm, …).
[ Info: Training machine(:knn, …).
[ Info: Training machine(:dt, …).
[ Info: Training machine(:svm, …).
[ Info: Training machine(:knn, …).
[ Info: Training machine(:dt, …).
[ Info: Training machine(:svm, …).
[ Info: Training machine(:knn, …).
[ Info: Training machine(:dt, …).
[ Info: Training machine(:svm, …).
[ Info: Training machine(:knn, …).
[ Info: Training machine(:dt, …).
[ Info: Training machine(:svm, …).
[ Info: Training machine(:knn, …).
[ Info: Training machine(:dt, …).
[ Info: Training machine(:svm, …).
[ Info: Training machine(:knn, …).
[ Info: Training machine(:dt, …).
[ Info: Training machine(:pca, …).
[ Info: Training machine(:pca, …).
[ Info: Training machine(:pca, …).
[ Info: Training machine(: